In [34]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/MyDrive/Sports Analytics/Data Sets')
os.getcwd()
os.listdir()





Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


['pff_2025_season - Copy(in) (1).csv', 'CU_2025_Season.csv']

In [46]:
import pandas as pd
import numpy as np
sports = pd.read_csv('pff_2025_season - Copy(in) (1).csv', index_col=0)

/tmp/ipython-input-347086503.py:3: DtypeWarning: Columns (49,50,61,88,89,97,98,115,122,123,124,128,136,157,158,189,205,206) have mixed types. Specify dtype option on import or set low_memory=False.
  sports = pd.read_csv('pff_2025_season - Copy(in) (1).csv', index_col=0)


#Visualization

In [47]:
#sports.head()
sports.info()
#sports.columns

<class 'pandas.core.frame.DataFrame'>
Index: 105440 entries, 6309366 to 6545480
Columns: 211 entries, pff_GAMEID to pff_EXPECTED_POINTS_ADDED_LEGACY
dtypes: bool(2), float64(36), int64(38), object(135)
memory usage: 169.1+ MB


#Data Cleaning

In [48]:
#Filtering to only 4th down
sports= sports[sports['pff_DOWN']==4]
#Removing Garbage time
sports = sports[sports['pff_GARBAGETIME'] != 1]
#Filling null values in pff_FIRST_DOWN_GAINED to 0
sports['pff_FIRST_DOWN_GAINED'] = sports['pff_FIRST_DOWN_GAINED'].fillna(0)

In [49]:
#filter for relevant columns
sports = sports[[
"pff_GAMEID", # Add pff_GAMEID back to the DataFrame
"pff_QUARTER",
 "pff_SORTORDER",
"pff_CLOCK",
"pff_DOWN",
 "pff_TRICKPLAY",
"pff_DISTANCE",
"pff_SPECIALTEAMSTYPE",
"pff_YARDS_TO_GOAL_LINE",
"pff_FIELDPOSITION",
"pff_OFFTEAM",
"pff_DEFTEAM",
"pff_OFFSCORE",
"pff_DEFSCORE",
"pff_SCOREDIFFERENTIAL",
"pff_EXPECTED_POINTS",
"pff_EXPECTED_POINTS_ADDED",
"pff_RUNPASS",
"pff_PASSRESULT",
"pff_GAINLOSS",
"pff_GAINLOSSNET",
"pff_FIRST_DOWN_GAINED",
"pff_TOUCHDOWN",
"pff_TURNOVER",
"pff_OFFSUCCESS",
"pff_DEF_TIMEOUTS_REMAINING",
"pff_OFF_TIMEOUTS_REMAINING"
]]

In [50]:
#Null values in this column represent a penalty, timeout, or kneel. Not an actual play
sports = sports[sports['pff_RUNPASS'].notna()]

In [51]:
sports['pff_RUNPASS'].unique()

array(['R', 'P', 'X'], dtype=object)

#4th Down conversion Success

In [52]:
#Filters 4th down plays to run, pass and trick play. Trick play is denoted as X in pff_RUNPASS
sports = sports[(sports['pff_RUNPASS'].isin(['R','P'])) | ((sports["pff_RUNPASS"] == "X") & (sports['pff_TRICKPLAY']==1))]

In [53]:
first_down_pct = sports["pff_FIRST_DOWN_GAINED"].mean() * 100
avg_yards_to_go = sports["pff_DISTANCE"].mean()
avg_yards = sports["pff_GAINLOSS"].mean()

print(f"First Down Percentage: {first_down_pct:.2f}%")
print(f"Average yards to first down:{avg_yards_to_go: .2f}")
print(f"Average yards gained:{avg_yards: .2f}")

First Down Percentage: 54.53%
Average yards to first down: 3.88
Average yards gained: 5.88


#Winning vs. going for it

In [ ]:
sports = sports[(sports['pff_DISTANCE']<=3.88)]
sports = sports[(sports['pff_QUARTER']==4)]

In [ ]:
# Make sure plays are in order within each game
sports_sorted = sports.sort_values(["pff_GAMEID", "pff_SORTORDER"])

# Get the last play per game for final score comparison
final_scores = (
    sports_sorted.groupby("pff_GAMEID", as_index=False)
    .agg({
        "pff_OFFTEAM": "last",
        "pff_DEFTEAM": "last",
        "pff_OFFSCORE": "last",
        "pff_DEFSCORE": "last"
    })
)

# Add a win flag for the offensive team
final_scores["offense_win_flag"] = (
    final_scores["pff_OFFSCORE"] > final_scores["pff_DEFSCORE"]
).astype(int)


In [ ]:
sports = sports.merge(
    final_scores[["pff_GAMEID", "offense_win_flag"]],
    on="pff_GAMEID",
    how="left"
)
